In [1]:
import tweepy
import pandas as pd
import numpy as np
import math
from datetime import datetime
import re
import emoji
import xlwt
import language_tool_python

In [2]:
lang_tool = language_tool_python.LanguageTool('en-US')

In [3]:
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 30

In [3]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [4]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])

    return clean_text

In [5]:
def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)

In [6]:
consumer_key = "###########"
consumer_secret = "###########"
access_token = "###########""
access_token_secret = "###########"

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
usernames = ["Wendys", "Skittles", "tacobell", "Pringles", "MoonPie",
             "HomeDepot", "Sears", "TD_Canada", "WestJet",
             "Applebees", "AEO", "BananaRepublic", "Bojangles", "krispykreme",
             "KayJewelers", "tide", "Chevron", "saveonfoods", "BMW",
             "DennysDiner","ChipotleTweets" ,"PopTartsUS", " hotpockets", "steak_umm"]
num_tweets = [4500, 2000, 1100, 1400, 1200,
              150, 1200, 750, 900,
              2000, 120, 120, 170, 300,
              1000, 1600, 400, 300, 600
              3000, 4000, 4000, 4000, 3000]

In [ ]:
usernames = ["DennysDiner","ChipotleTweets" ,"PopTartsUS", " hotpockets", "steak_umm"]
num_tweets = [3000, 4000, 4000, 4000, 3000]

In [8]:
usernames = ["DennysDiner"]
num_tweets = [10]

In [9]:
# This cell fetchs twitter data for all usernames and save in a dataframe "tweets_df_all"
first_loop = 1
num_tweets_i = 0

for username in usernames:
    
    count = num_tweets[num_tweets_i]
    num_tweets_i += 1
    twitter_name = api.get_user(screen_name = username).name

    try:     
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,id=username, tweet_mode='extended').items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.id, tweet.created_at, tweet.full_text, tweet.favorite_count,
                        tweet.retweet_count, tweet.retweeted, tweet.entities, 
                        tweet.in_reply_to_user_id, tweet.source] for tweet in tweets]

        # Creation of dataframe from tweets list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)

    tweets_df.columns = ['tweet_id', "time_tweeted", "text", "num_hearts",
                  "num_retweets", "retweeted_by_user", "entities", "reply_to_user_id",
                  "source"]
    tweets_df.insert(loc = 0, column = 'handle', value = 'NA') 
    tweets_df.insert(loc = 1, column = 'twitter_name', value = 'NA') 

    for i in range(len(tweets_df)):
        entity = tweets_df.at[i,'entities']

        #Checks if media is attached to tweet and reports its type (note videos also count as "photo")
        if 'media' in entity:
            if 'type' in entity['media'][0]:
                media = entity['media'][0]['type']
                media_url = entity['media'][0]['media_url_https']
                media_link = entity['media'][0]['url']
        else:
            media = "NA"
            media_url = "NA"
            media_link = "NA"

        # Checks if tweet is a reply by seeing if reply ID exists
        if pd.isnull(tweets_df.at[i,'reply_to_user_id']):
            tweets_df.at[i, 'is_reply'] = False
        else:
            tweets_df.at[i, 'is_reply'] = True

        # Checks if tweet is a retweet or not by looking at start of text
        if tweets_df.at[i,'text'][:3] == "RT ":
            tweets_df.at[i, 'is_other_user_rt'] = True
            tweets_df.at[i,'text'] = tweets_df.at[i,'text'][3:]
        else:
            tweets_df.at[i, 'is_other_user_rt'] = False

        #Retweets
        if not entity['hashtags']:
            num_hashtags = 0
            hashtags = "NA"
        else:
            hashtags = ""
            num_hashtags = 0
            for g in entity['hashtags']:
                hashtags= hashtags + g['text']
                hashtags= hashtags + ", "
                num_hashtags+=1
            hashtags = hashtags[:-2]

        #Mentions
        if not entity['user_mentions']:
            num_mentions = 0
        else:
            num_mentions = 0
            for g in entity['user_mentions']:
                num_mentions+=1

        #Text with no hashtags, mentions, links, emojis
        text_only_1 = re.sub(r"\$\w+[,]|\@\w+|[,]\$\w+", "", tweets_df.at[i,'text']) #removes '@'s
        text_only_2 = re.sub(r"\$\w+[,]|\#\w+|[,]\$\w+", "", text_only_1) #removes hashtags
        text_only_3 = re.sub(r'http\S+', '', text_only_2) #removes https links
        text_only_4 = text_only_3.replace("\n", " ") #remove \n newlines
        text_only_5 = give_emoji_free_text(deEmojify(text_only_4))
        text_only_6 = text_only_5.strip(':').lstrip()
        text_only_7 = text_only_6.lstrip().rstrip() #lstrip removes leading whitespace, rstrip trailing
        
        #Number of grammar errors per character
        num_words = len(text_only_7.split())
        if (num_words > 3):
            errors_per_word = len(lang_tool.check(text_only_7)) / num_words
        else:
            errors_per_word = 0

        #Adding columns
        tweets_df.at[i, 'media'] = media
        tweets_df.at[i, 'hashtags'] = hashtags
        tweets_df.at[i, 'num_hashtags'] = int(num_hashtags)
        tweets_df.at[i, 'num_mentions'] = int(num_mentions)
        tweets_df.at[i, 'media_url'] = media_url
        tweets_df.at[i, 'media_link'] = media_link
        tweets_df.at[i, 'tweet_age_days'] = (datetime.utcnow() - tweets_df.at[i,'time_tweeted']).days
        tweets_df.at[i, 'tweet_length'] = len(tweets_df.at[i,'text'])
        tweets_df.at[i, 'emojis'] = extract_emojis(tweets_df.at[i,'text'])
        tweets_df.at[i, 'num_emojis'] = len(extract_emojis(tweets_df.at[i,'text']))
        tweets_df.at[i, 'tweet_text_only'] = text_only_7
        tweets_df.at[i, 'twitter_name'] = twitter_name
        tweets_df.at[i, 'handle'] = username
        tweets_df.at[i, 'grammar_errors_per_word'] = errors_per_word
        tweets_df.at[i, 'num_words'] = num_words

    tweets_df = tweets_df[['handle', 'twitter_name', 'is_other_user_rt', 'is_reply',
                           'text', 'num_hearts', "num_retweets", 'tweet_length','time_tweeted',
                           'tweet_age_days', 'hashtags', 'num_hashtags', 'num_mentions', 'retweeted_by_user',
                           'emojis', 'num_emojis', 'media', 'media_url', 'media_link', 'tweet_text_only',
                           'grammar_errors_per_word', 'num_words', 'entities', 'reply_to_user_id',
                           'tweet_id', 'source']]
    
    if first_loop == 1:
        first_loop = 0
        tweets_df_all = tweets_df.copy(deep = True)
        del tweets_df
    else:
        tweets_df_all = pd.concat([tweets_df_all, tweets_df])
        del tweets_df

tweets_df_all = tweets_df_all.reset_index(drop=True)

In [13]:
tweets

In [12]:
tweets_df_all.head(3)

,handle,twitter_name,is_other_user_rt,is_reply,text,num_hearts,num_retweets,tweet_length,time_tweeted,tweet_age_days,...,media,media_url,media_link,tweet_text_only,grammar_errors_per_word,num_words,entities,reply_to_user_id,tweet_id,source
0,DennysDiner,Denny’s,False,False,nannerpuss? https://t.co/vPgHG5i3vY,187,18,35.0,2021-02-23 19:29:49,2.0,...,NA,NA,NA,nannerpuss?,0.000000,1.0,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1364296423362158594,Twitter Web App
1,DennysDiner,Denny’s,False,True,@theunitedbacon NEED to know more!!,13,1,35.0,2021-02-23 16:27:49,3.0,...,NA,NA,NA,NEED to know more!!,0.000000,4.0,"{'hashtags': [], 'symbols': [], 'user_mentions...",7.852077e+17,1364250621038243843,Twitter for iPhone
2,DennysDiner,Denny’s,False,True,@Raichudvd13 okay but like are they still alive,12,0,47.0,2021-02-23 16:27:25,3.0,...,NA,NA,NA,okay but like are they still alive,0.142857,7.0,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.183911e+18,1364250522258001924,Twitter for iPhone


In [ ]:
#remove reply tweets
tweets_df_all_no_replies = tweets_df_all.loc[tweets_df_all['is_reply'] == False]
tweets_df_all_no_replies = tweets_df_all_no_replies.loc[tweets_df_all_no_replies['tweet_age_days'] > 0]
tweets_df_all_no_replies = tweets_df_all_no_replies.sample(frac=1)

#calculate avg performance per account
avg_likes = tweets_df_all_no_replies.groupby('handle', as_index=False)['num_hearts'].mean()
avg_likes.columns = ['handle', 'handle_avg_hearts']
avg_retweets = tweets_df_all_no_replies.groupby('handle', as_index=False)['num_retweets'].mean()
avg_retweets.columns = ['handle', 'handle_avg_rts']

tweets_df_all_no_replies = tweets_df_all_no_replies.merge(avg_likes, on='handle', how='left')
tweets_df_all_no_replies = tweets_df_all_no_replies.merge(avg_retweets, on='handle', how='left')

tweets_df_all_no_replies['prptn_rts_vs_acc_avg'] = tweets_df_all_no_replies['num_retweets']/tweets_df_all_no_replies['handle_avg_rts']
tweets_df_all_no_replies['prptn_hrts_vs_acc_avg'] = tweets_df_all_no_replies['num_hearts']/tweets_df_all_no_replies['handle_avg_hearts']

#saves dataset
#tweets_df_all_no_replies.to_excel('twitter_data_raw.xls', index=False)